In [1]:
import pandas as pd
import numpy as np
import matplotlib as plt
from sklearn.model_selection import train_test_split

testData = pd.read_csv("test.csv")
testY = pd.read_csv("output.csv")
trainData = pd.read_csv("train.csv")

In [2]:
#Filling Missing BMI Data

temp = trainData['bmi'].mean() + trainData['bmi'].median()
temp /= 2
temp
trainData['bmi'] = trainData['bmi'].fillna(temp)


In [3]:
len( list( filter( lambda x: x == True , pd.isna(trainData['smoking_status']) ) ) )

13292

In [4]:
# trainData['smoking_status'] = trainData['bmi'].fillna("unknown")

trainDataNS = trainData[ trainData['smoking_status'].isna() ]
trainDataNS = trainDataNS.drop( columns=['smoking_status'] )
trainDataS = trainData[ trainData['smoking_status'].notna() ]

testDataNS = testData[ testData['smoking_status'].isna() ]
testDataNS = testDataNS.drop( columns=['smoking_status'] )
testDataS = testData[ testData['smoking_status'].notna() ]

print( testDataS.shape )
print( testDataNS.shape )
print( trainDataS.shape )
print( trainDataNS.shape )

(12850, 11)
(5751, 10)
(30108, 12)
(13292, 11)


In [5]:
yTrainDataNS = trainDataNS['stroke']
xTrainDataNS = trainDataNS.drop( columns=['stroke'] )
#xTrainDataNS.head()

yTrainDataS = trainDataS['stroke']
xTrainDataS = trainDataS.drop( columns=['stroke'] )
#xTrainDataS.head()

In [6]:
from sklearn.linear_model import LogisticRegression as LR
from sklearn.tree import DecisionTreeClassifier
clf = DecisionTreeClassifier(random_state=0)
lr = LR()

In [7]:
xTrainDataNS = pd.get_dummies( xTrainDataNS, columns=['gender', 'ever_married', 'work_type' , 'Residence_type']
                            , prefix=['sex', 'married', 'work_type' , 'res_type'] )
xTrainDataNS.head()

,id,age,hypertension,heart_disease,avg_glucose_level,bmi,sex_Female,sex_Male,sex_Other,married_No,married_Yes,work_type_Govt_job,work_type_Never_worked,work_type_Private,work_type_Self-employed,work_type_children,res_type_Rural,res_type_Urban
0,30669,3.0,0,0,95.12,18.0,0,1,0,1,0,0,0,0,0,1,1,0
2,16523,8.0,0,0,110.89,17.6,1,0,0,1,0,0,0,1,0,0,0,1
4,46136,14.0,0,0,161.28,19.1,0,1,0,1,0,0,1,0,0,0,1,0
5,32257,47.0,0,0,210.95,50.1,1,0,0,0,1,0,0,1,0,0,0,1
10,10460,79.0,0,0,77.08,35.0,1,0,0,0,1,1,0,0,0,0,0,1


In [19]:
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import SMOTE
from collections import Counter
sm=SMOTE()
print("1st Check",Counter(yTrainDataNS))
sm_x,sm_y=sm.fit_sample(xTrainDataNS,yTrainDataNS)
print("1st Check",Counter(sm_y))
x_train, x_test, y_train, y_test = train_test_split(sm_x,sm_y, test_size=0.25, random_state=0)



lr.fit(x_train, y_train)
clf.fit(x_train,y_train)

1st Check Counter({0: 13147, 1: 145})
1st Check Counter({0: 13147, 1: 13147})


DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=0,
            splitter='best')

In [20]:
print(x_test.shape)
print(y_test.shape)

(6574, 18)
(6574,)


In [21]:
predictions = lr.predict(x_test)
predictions_Tree=clf.predict(x_test)

print(lr.score(x_test, y_test))
print(clf.score(x_test, y_test))

0.8553392150897475
0.9864618192881046


In [22]:
from sklearn import metrics

print(metrics.confusion_matrix(y_test, predictions))
print(metrics.confusion_matrix(y_test,predictions_Tree))
#MCC is better metric along with F-Score


[[2638  634]
 [ 317 2985]]
[[3216   56]
 [  33 3269]]


In [23]:
#Logistic 
metrics.matthews_corrcoef(y_test, predictions, sample_weight=None)

0.7138756435424061

In [24]:
#Tree
metrics.matthews_corrcoef(y_test, predictions_Tree, sample_weight=None)

0.9729460313870295

In [25]:
#Logistic
metrics.f1_score(y_test, predictions, labels=None, pos_label=1, average='binary', sample_weight=None)

0.8625921109666232

In [26]:
#Tree
metrics.f1_score(y_test, predictions_Tree, labels=None, pos_label=1, average='binary', sample_weight=None)

0.9865700920476838

In [16]:
testDataS.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status
0,36306,Male,80.0,0,0,Yes,Private,Urban,83.84,21.1,formerly smoked
1,61829,Female,74.0,0,1,Yes,Self-employed,Rural,179.50,26.0,formerly smoked
4,40801,Female,63.0,0,0,Yes,Govt_job,Rural,83.57,27.6,never smoked
5,9348,Female,66.0,1,0,Yes,Private,Urban,219.98,32.2,never smoked
7,60512,Male,46.0,0,0,Yes,Govt_job,Urban,120.80,32.5,never smoked


In [17]:
trainDataS.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
1,30468,Male,58.0,1,0,Yes,Private,Urban,87.96,39.2,never smoked,0
3,56543,Female,70.0,0,0,Yes,Private,Rural,69.04,35.9,formerly smoked,0
6,52800,Female,52.0,0,0,Yes,Private,Urban,77.59,17.7,formerly smoked,0
7,41413,Female,75.0,0,1,Yes,Self-employed,Rural,243.53,27.0,never smoked,0
8,15266,Female,32.0,0,0,Yes,Private,Rural,77.67,32.3,smokes,0
